[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/openai_agenticrag.ipynb)

In [ ]:
!pip install indox
!pip install openai
!pip install chromadb
!pip install duckduckgo-search

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


## Agentic RAG
Here, we will explore how to work with Agentic RAG. We are using OpenAI and we should set our OPENAI_API_KEY as an environment variable.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Creating an instance of IndoxRetrievalAugmentation
You must first create an instance of IndoxRetrievalAugmentation class. This instance will allow you to access the methods and properties defined within the class, enabling the augmentation and retrieval functionalities.

In [2]:
from indox import IndoxRetrievalAugmentation
from indox.llms import OpenAi
from indox.embeddings import OpenAiEmbedding
from indox.data_loader_splitter import SimpleLoadAndSplit

Create OpenAi model as LLM_model and OpenAiEmbedding as Embedding model and using them to generate response.

In [3]:
indox = IndoxRetrievalAugmentation()
llm_model = OpenAi(api_key=OPENAI_API_KEY,model="gpt-3.5-turbo-0125")
embed = OpenAiEmbedding(api_key=OPENAI_API_KEY,model="text-embedding-3-small")

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            
INFO: Initializing OpenAi with model: gpt-3.5-turbo-0125
INFO: OpenAi initialized successfully
INFO: Initialized OpenAI embeddings with model: text-embedding-3-small


In [4]:
indox.__version__

'0.1.13'

### You can download the file from the below address 

In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

## Preprocess Data
using SimpleLoadAndSplit class to preprocess text data from a file, split text into chunks

In [6]:
loader_splitter = SimpleLoadAndSplit(file_path="sample.txt",remove_sword=False)

INFO: UnstructuredLoadAndSplit initialized successfully


In [7]:
docs = loader_splitter.load_and_chunk()

INFO: Starting processing
INFO: Created initial document elements
INFO: Completed chunking process
INFO: Successfully obtained all documents


## Create ChromaVectoreStore instance
Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [8]:
from indox.vector_stores import Chroma

# Define the collection name within the vector store
collection_name = "sample"

# Create a ChromaVectorStore instance
db = Chroma(collection_name=collection_name, embedding_function=embed)


INFO: Connection to the vector store database established successfully


store the chunks in the vector store that was set up previously.

In [9]:
db.add(docs=docs)

INFO: Storing documents in the vector store
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


## Retrieve relevant information by question-answering model
At this step we are using QuestionAnswer model and try to retrieve the answer just by our file and without any agent

In [10]:
query = "Where does messi plays right now?"
retriever = indox.QuestionAnswer(vector_database=db,llm=llm_model,top_k=3)

In [11]:
retriever.invoke(query)

INFO: Retrieving context and scores from the vector database
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Generating response
INFO: Response generated successfully
INFO: Query answered successfully


"I'm sorry, but the given context does not contain any information about Lionel Messi's current football club."

## Retrieve information by using Agnet
Here we are using Agent to retrieve answer. As you can see, our last try was unsuccessful but now after the agent couldn't find the answer it started to search on the internet.
Note: to be more familiar with AgenticRAG pleas read [this page]("https://docs.osllm.ai/agenticRag.html")

In [12]:
agent = indox.AgenticRag(llm=llm_model,vector_database=db,top_k=3)
agent.run(query)

INFO: Generating response
ERROR: Error generating response: Request timed out.
ERROR: Error generating response: Request timed out.
INFO: Generating response
INFO: Response generated successfully
INFO: Not relevant doc
INFO: Generating response
INFO: Response generated successfully
INFO: Not relevant doc
INFO: Generating response
INFO: Response generated successfully
INFO: Not relevant doc
INFO: No Relevant document found, Start web search
INFO: No Relevant Context Found, Start Searching On Web...
INFO: Answer Base On Web Search
INFO: Answering question
INFO: Generating response
INFO: Response generated successfully
INFO: Check For Hallucination In Generated Answer Base On Web Search
INFO: Checking hallucination for answer
INFO: Generating response
INFO: Response generated successfully
INFO: Hallucination detected, Regenerate the answer...
INFO: Answering question
INFO: Generating response
INFO: Response generated successfully


"Lionel Messi currently plays for Major League Soccer's Inter Miami CF."